# Data Setup

In [1]:
import numpy as np
import pandas as pd
import os

## Create Star Data
Parse through the directory with all the head shots and create a dataframe of all the stars and their show

In [2]:
# Create the dictionary
star_data = []

In [3]:
# Get list of all stars
files = os.listdir("../Assets/Headshots")
for f in files:
    # get the show
    show = f[0:f.index("_")]
    
    # Get the star
    star = f[f.index("_") + 1:f.index(".")]
    
    # Get the status (for now made up)
    status = "In the hunt"
    
    # Add to data
    star_data.append({"show":show, "star":star, "status":status, "file":f})

In [4]:
# Make the data frame
star_df = pd.DataFrame(star_data)

In [5]:
star_df.head()

,show,star,status,file
0,sur,rob,In the hunt,sur_rob.jpg
1,sur,adam,In the hunt,sur_adam.jpg
2,tc,kevin,In the hunt,tc_kevin.jpg
3,tc,melissa,In the hunt,tc_melissa.jpg
4,sur,danni,In the hunt,sur_danni.jpg


In [20]:
# Write to js
s = "var starData = " + star_df.to_json(orient='records') + ';'
out_file = open("starData.js", "w+")
out_file.write(s)
out_file.close()

In [8]:
# Write to csv
star_df.sort_values("show").to_csv("star_data.csv")
# star_df.to_csv("star_data.csv")

## Teams and Draft Picks

In [26]:
draft = pd.read_excel("draft_results1-17-20.xlsx")
draft.head()

,Name,Show,PickedBy,Eliminated
0,Shiann,Bachelor,Alex C,NaN
1,Delvin,Project Runway,Alex C,NaN
2,Natalie,Survivor,Alex C,NaN
3,Rob,Survivor,Alex C,NaN
4,Gregory,Top Chef,Alex C,NaN


In [27]:
# Add a column for score
draft["Score"] = 0

In [29]:
s = "var userData = " + draft.to_json(orient='records') + ';'
out_file = open("userData.js", "w+")
out_file.write(s)
out_file.close()

# Combining Data

In [51]:
# Read the star Data
star = pd.read_csv("star_data.csv").iloc[:, 1:]
star.rename(columns={"show":"Show", "star":"Name"}, inplace=True)
star.head()

,Show,Name,status,file
0,bach,Sydney,In the hunt,bach_sydney.png
1,bach,Jasmine,In the hunt,bach_jasmine.png
2,bach,Natasha,In the hunt,bach_natasha.png
3,bach,Kelsey,In the hunt,bach_kelsey.png
4,bach,Victoria F,In the hunt,bach_victoriaF.png


In [52]:
# Make a mapping from show acronym to full show name
show_map = {
    "bach": "The Bachelor",
    "tc" : "Top Chef",
    "kbc" : "Kids Baking Championship",
    "sur" : "Survivor",
    "pr": "Project Runway"
}

In [53]:
# Apply this to the show column
star = star.replace({"Show":show_map})
star.head()

,Show,Name,status,file
0,The Bachelor,Sydney,In the hunt,bach_sydney.png
1,The Bachelor,Jasmine,In the hunt,bach_jasmine.png
2,The Bachelor,Natasha,In the hunt,bach_natasha.png
3,The Bachelor,Kelsey,In the hunt,bach_kelsey.png
4,The Bachelor,Victoria F,In the hunt,bach_victoriaF.png


In [54]:
# Read in the draft data
draft = pd.read_excel("draft_results1-17-20.xlsx")
draft.replace({"Bachelor":"The Bachelor"}, inplace=True)
draft.head()

,Name,Show,PickedBy,Eliminated
0,Shiann,The Bachelor,Alex C,NaN
1,Delvin,Project Runway,Alex C,NaN
2,Natalie,Survivor,Alex C,NaN
3,Rob,Survivor,Alex C,NaN
4,Gregory,Top Chef,Alex C,NaN


In [55]:
# join them
joined = pd.merge(star, draft, left_on=['Name', 'Show'], right_on=['Name', 'Show'], how='left')
joined

,Show,Name,status,file,PickedBy,Eliminated
0,The Bachelor,Sydney,In the hunt,bach_sydney.png,Alex G,NaN
1,The Bachelor,Jasmine,In the hunt,bach_jasmine.png,Jessica,NaN
2,The Bachelor,Natasha,In the hunt,bach_natasha.png,Marie,NaN
3,The Bachelor,Kelsey,In the hunt,bach_kelsey.png,Shelley,NaN
4,The Bachelor,Victoria F,In the hunt,bach_victoriaF.png,Andrew,NaN
...,...,...,...,...,...,...
68,Top Chef,Karn,In the hunt,tc_karen.jpg,NaN,NaN
69,Top Chef,Bryan V,In the hunt,tc_bryan.jpg,Kenny,NaN
70,Top Chef,Stephanie,In the hunt,tc_stephanie.jpg,Shelley,NaN
71,Top Chef,Jennifer,In the hunt,tc_jennifer.jpg,Maddie,NaN


In [56]:
# Clean them up
joined.drop("status", axis=1, inplace=True)
joined["Eliminated"] = 0
joined["Score"] = 0

In [57]:
# Ouptut
s = joined.to_json(orient='records')
f = open("data.js", 'w+')
f.write("var data = " + s + ";")
f.close()